# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
# word net installation:

# unmark if you want to use and need to install
!pip install wn
!python -m wn download omw-he:1.4


Cached file found: C:\Users\nivoz\.wn_data\downloads\7ecf10e89326bc0ac26ad94b40fe60a7b6ac3302

Checking C:\Users\nivoz\AppData\Local\Temp\tmpe_f2gnoc\omw-he\omw-he.xml
Skipping omw-he:1.4 (Hebrew Wordnet); already added



In [4]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
!pip install hebrew_tokenizer

In [6]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

C:\Users\nivoz\assignment5-text_analysis


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [7]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [8]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [9]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [10]:
def splitingFeaturesLabels(data):
    X_train = data.iloc[:,:-1]
    y_train = data.iloc[:, -1:]
    return X_train, y_train

features, labels = splitingFeaturesLabels(df_train)

In [11]:
import re

def preprocess_hebrew_text(text):
    # Remove non-Hebrew characters
    text = re.sub(r"[^\u0590-\u05FF\s]+", "", text)
    
    # Remove diacritics and vowel points
    text = re.sub(r"[\u05B0-\u05C7]", "", text)
    
    # Normalize characters
    # Example: Replace final kaf (ך) with regular kaf (כ)
    text = re.sub(r"\u05DA", "\u05DB", text)
    
    # Remove punctuation and special characters
    text = re.sub(r"[^\w\s]+", "", text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

features['story'] = features['story'].apply(preprocess_hebrew_text)

In [12]:
def dataToVertorizedDf(data):
    vectorizer = TfidfVectorizer(max_features = 7000, ngram_range = (1,2), sublinear_tf = True)
    vectorizer_matrix = vectorizer.fit_transform(data['story'])
    vectorizer_df = pd.DataFrame(data = vectorizer_matrix.toarray(), columns = vectorizer.get_feature_names())
    return vectorizer_df

vectorized_df = dataToVertorizedDf(features)

#splitting the data to train and test (80/20 ratio)
X_train, X_test, y_train, y_test = train_test_split(vectorized_df, labels, test_size = 0.2, random_state = 42)

In [13]:
linearSVC = LinearSVC()

# Define the parameter grid to search over
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'loss': ['hinge', 'squared_hinge'],
    'dual': [False],
    'tol': [1e-3, 1e-4, 1e-5]
}

# Create a GridSearchCV object
grid_search = GridSearchCV(linearSVC, param_grid, cv=5, scoring='f1_macro')

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters found by the grid search
print('Best hyperparameters:', grid_search.best_params_)

# Print the mean cross-validated F1 score of the best estimator
print('Best accuracy score:', grid_search.best_score_)

best_linearSVC = grid_search.best_estimator_
y_pred = best_linearSVC.predict(X_test)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred, average='macro')
print("F1 score:", f1)

GridSearchCV(cv=5, estimator=LinearSVC(),
             param_grid={'C': [0.1, 1, 10], 'dual': [False],
                         'loss': ['hinge', 'squared_hinge'],
                         'penalty': ['l1', 'l2'],
                         'tol': [0.001, 0.0001, 1e-05]},
             scoring='f1_macro')

Best hyperparameters: {'C': 10, 'dual': False, 'loss': 'squared_hinge', 'penalty': 'l1', 'tol': 1e-05}
Best accuracy score: 0.6349884642207704
F1 score: 0.6711672473867596


In [14]:
vectorized_test_df = dataToVertorizedDf(df_test)
n = best_linearSVC.predict(vectorized_test_df)

In [16]:
df_test['gender'] = n

In [17]:
df_test.head()
df_test.tail()

,test_example_id,story,gender
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...,m
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""...",m
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...,m
3,3,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני...",f
4,4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...,m


,test_example_id,story,gender
318,318,"בשנה האחרונה הרגשתי די תקוע בעבודה, השגרה הפכה...",f
319,319,אני ואילן חברים טובים מזה 20 שנה תמיד חלמנו לפ...,m
320,320,מידי יום שישי אני נוהג לנסוע בתחבורה ציבורית ס...,f
321,321,"לפני מספר חודשים, בשיא התחלואה של הגל השני, עמ...",f
322,322,היום בו דיווחתי על גניבה של האוטו שלי. בוקר אח...,m


In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

knn = KNeighborsClassifier()

# Define the parameter grid to search over
param_grid = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': [1, 2]
}

# Create a GridSearchCV object
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='f1_macro')

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters found by the grid search
print('Best hyperparameters:', grid_search.best_params_)

# Print the mean cross-validated F1 score of the best estimator
print('Best accuracy score:', grid_search.best_score_)

best_knn = grid_search.best_estimator_
y_pred = best_knn.predict(X_test)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred, average='macro')
print("F1 score:", f1)

vectorized_test_df = dataToVertorizedDf(df_test)
n = best_knn.predict(vectorized_test_df)


GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': [3, 5, 7], 'p': [1, 2],
                         'weights': ['uniform', 'distance']},
             scoring='f1_macro')

Best hyperparameters: {'algorithm': 'auto', 'n_neighbors': 3, 'p': 2, 'weights': 'distance'}
Best accuracy score: 0.6396881563185305
F1 score: 0.6690575811914724


In [20]:
df_test.head()
df_test.tail()

,test_example_id,story,gender
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...,m
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""...",m
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...,m
3,3,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני...",f
4,4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...,m


,test_example_id,story,gender
318,318,"בשנה האחרונה הרגשתי די תקוע בעבודה, השגרה הפכה...",f
319,319,אני ואילן חברים טובים מזה 20 שנה תמיד חלמנו לפ...,m
320,320,מידי יום שישי אני נוהג לנסוע בתחבורה ציבורית ס...,f
321,321,"לפני מספר חודשים, בשיא התחלואה של הגל השני, עמ...",f
322,322,היום בו דיווחתי על גניבה של האוטו שלי. בוקר אח...,m


In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

decision_tree = DecisionTreeClassifier()

# Define the parameter grid to search over
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Create a GridSearchCV object
grid_search = GridSearchCV(decision_tree, param_grid, cv=5, scoring='f1_macro')

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters found by the grid search
print('Best hyperparameters:', grid_search.best_params_)

# Print the mean cross-validated F1 score of the best estimator
print('Best accuracy score:', grid_search.best_score_)

best_decision_tree = grid_search.best_estimator_
y_pred = best_decision_tree.predict(X_test)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred, average='macro')
print("F1 score:", f1)

vectorized_test_df = dataToVertorizedDf(df_test)
n = best_decision_tree.predict(vectorized_test_df)


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 5, 10, 20],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='f1_macro')

Best hyperparameters: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2}
Best accuracy score: 0.6066889647303682
F1 score: 0.5578828828828829


In [24]:
df_test.head()
df_test.tail()

,test_example_id,story,gender
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...,m
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""...",m
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...,m
3,3,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני...",f
4,4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...,m


,test_example_id,story,gender
318,318,"בשנה האחרונה הרגשתי די תקוע בעבודה, השגרה הפכה...",f
319,319,אני ואילן חברים טובים מזה 20 שנה תמיד חלמנו לפ...,m
320,320,מידי יום שישי אני נוהג לנסוע בתחבורה ציבורית ס...,f
321,321,"לפני מספר חודשים, בשיא התחלואה של הגל השני, עמ...",f
322,322,היום בו דיווחתי על גניבה של האוטו שלי. בוקר אח...,m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [25]:
df_predicted.to_csv('classification_results.csv',index=False)

NameError: name 'df_predicted' is not defined